In [8]:
!pip install markovify

     |████████████████████████████████| 235 kB 5.2 MB/s 
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18628 sha256=1d977d0dc8dda126066ab08a728b7beadea67c89d42b59f9f9d96ab3fd9ac104
  Stored in directory: /root/.cache/pip/wheels/36/c5/82/11125c5a7dadec27ef49ac2b3a12d3b1f79ff7333c92a9b67b
Successfully built markovify


In [137]:
import re
import markovify
import spacy
import json

In [21]:
#чистим тексты
for i in range (4):
  filename = 'mhb' + str(i+1) + '.txt'
  with open (filename, 'r', encoding="UTF-8") as f:
    mhb = f.read()
  mhb = mhb.replace('\ufeff', '')
  mhb = re.sub(r'\nSECTION \w+\n', ' ', mhb) #названия частей
  mhb = re.sub(r'BOOK \w+\n', '', mhb) #названия книги
  mhb = mhb.replace('\n', ' ') # в одну длинную строку
  mhb = re.sub(r'\s[A-Z]{2,}', '', mhb) #и остальные названия капсом убираем
  mhb = re.sub(r'[0-9]{1,}', '', mhb) #и всякие цифры
  mhb = re.sub(r'\s{2,}', ' ', mhb) #чтобы максимум был 1 пробел между
  mhb = mhb.replace('[]', ' ')
  mhb = re.sub(r'[()]', '', mhb)
  if i+1 == 1:
    mhb1 = mhb
  elif i+1 == 2:
    mhb2 = mhb
  elif i+1 == 3:
    mhb3 = mhb
  else:
    mhb4 = mhb

In [26]:
nlp = spacy.load('en_core_web_sm')

In [136]:
#cтараемся научить модель учитывать части речи слов при генерации текстов,
#что, в теории, позволит ей строить более приличные с точки зрения синтаксиса
#предложения
class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

In [35]:
model1 = POSifiedText(mhb1, state_size = 3)

In [36]:
model2 = POSifiedText(mhb2, state_size = 3)

In [37]:
model3 = POSifiedText(mhb3, state_size = 3)

In [38]:
model4 = POSifiedText(mhb4, state_size = 3)

In [60]:
combo_model = markovify.combine([ model1, model2, model3, model4 ], [ 1, 1, 1, 1 ])

In [139]:
#обучение моделей занимает много времени, поэтому экспортируем готовые в .json
#сможем потом просто загружать их, а не обучать с нуля
combo_model_json = combo_model.to_json()

In [140]:
with open('combo_model.json', 'w') as outfile:
    json.dump(combo_model_json, outfile)

In [138]:
model1_json = model1.to_json()
model2_json = model2.to_json()
model3_json = model3.to_json()
model4_json = model4.to_json()

In [142]:
with open('model1.json', 'w') as outfile1:
    json.dump(model1_json, outfile1)
with open('model2.json', 'w') as outfile2:
    json.dump(model2_json, outfile2)
with open('model3.json', 'w') as outfile3:
    json.dump(model3_json, outfile3)
with open('model4.json', 'w') as outfile4:
    json.dump(model4_json, outfile4)